# Example 2: Coupled Transmon Qubits

In [1]:
%load_ext autoreload
%autoreload 2
import time

In [2]:
import json
import design as d
import names as n

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)

from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys

## Design assembly

In [3]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um")
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

n.add_design_variables_to_design(design, initial_design_variables)

In [4]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, group=n.GROUP_1)
    d.add_transmon_plus_resonator(design, group=n.GROUP_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, group=n.GROUP_1)
    d.add_chargeline(design, group=n.GROUP_2)

    gui.rebuild()
    gui.autoscale()


render_qiskit_metal_design(design, gui)

## MiniStudies

In [5]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

### Single resonator-qubit system
Useful when first tuning up subsystems

In [ ]:
def CoupledLineTee_mesh_names(comp_names):
    all_names_to_mesh = [f"prime_cpw_{comp_names}", f"second_cpw_{comp_names}"]
    return all_names_to_mesh

In [ ]:
MINI_STUDY_GROUP = n.GROUP_1
MINI_STUDY = ms.get_mini_study_qb_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS,
    meshing_map=[
        MeshingMap(component_class=CoupledLineTee, mesh_names=CoupledLineTee_mesh_names)
    ],
)

group_runs = 2  # 10
group_passes = 2  # 6
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
design_analysis.overwrite_parameters()

### Two qubit-resonator system

In [ ]:
MINI_STUDY = ms.get_mini_study_2qb_resonator_coupler()
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[n.GROUP_1, n.GROUP_2],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
    opt_target_coupler_freq=True,
)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_TWO_QB,
)

group_runs = 2
group_passes = 6
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
design_analysis.overwrite_parameters()

### Purcell decay into charge line

The T1 of the qubit is limited by its decay into the charge line. This ministudy will change the distance of the charge line from the transmon pocket to optimize a T1 limit of 1 ms. This value is computed with simple capacitance matrix simulations and a classical model, assuming that the qubit frequency has already been optimized.

In [6]:
close_ansys()

In [7]:
MINI_STUDY_GROUP = 1
MINI_STUDY = ms.get_mini_study_qb_charge_line(group=MINI_STUDY_GROUP)
opt_targets = ot.get_opt_targets_qb_charge_line(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

In [8]:
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CHARGE_LINE_DECAY,
)

[INFO|2025-03-03 15:22:53]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 03:23PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:23PM [load_ansys_project]: 	Opened Ansys App
INFO 03:23PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 03:23PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/lukassp/Documents/Ansoft/
	Project:   Project198
INFO 03:23PM [connect_design]: No active design found (or error getting active design).
INFO 03:23PM [connect]: 	 Connected to project "Project198". No design detected
03:23PM 00s WARNING [activate_ansys_design]: The design_name=get_mini_study_qb_charge_line was not in active project.  Desig

In [9]:
group_runs = 5
group_passes_cap = 5

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-03-03 15:23:02]: Updated_design_vars
{
    "cpw_width": "10 um",
    "cpw_gap": "6 um",
    "design_var_width_qubit_1": "400um",
    "design_var_width_qubit_2": "400um",
    "design_var_cl_pos_x_qubit_1": "-2600um",
    "design_var_cl_pos_y_qubit_1": "-1800um",
    "design_var_cl_pos_x_qubit_2": "-2600um",
    "design_var_cl_pos_y_qubit_2": "1800um",
    "design_var_length_resonator_1_capacitance": "20um",
    "design_var_length_resonator_2_capacitance": "20um",
    "design_var_lj_qubit_1": "12.1nH",
    "design_var_lj_qubit_2": "9.1nH",
    "design_var_cj_qubit_1": "0fF",
    "design_var_cj_qubit_2": "0fF",
    "design_var_length_resonator_1": "7500um",
    "design_var_length_resonator_2": "6500um",
    "design_var_length_coupler_1to2": "4000um",
    "design_var_coupl_length_qubit_1_resonator_1": "100um",
    "design_var_coupl_length_qubit_2_resonator_2": "100um",
    "design_var_coupl_length_resonator_1_tee": "170um",
    "design_var_coupl_length_resonator_2_tee": "160um"


### Optimize capacitance target

In [ ]:
MINI_STUDY_BRANCH = 1
MINI_STUDY = ms.get_mini_study_resonator_capacitance(group=MINI_STUDY_BRANCH)
opt_targets = ot.get_opt_target_capacitance(group=MINI_STUDY_BRANCH)

RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path="out/" + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CAPACITANCE,
)

In [ ]:
group_runs = 10
group_passes_cap = 2

for i in range(group_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

## View Optimization results

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

In [ ]:
close_ansys()